# 02 — Data Cleaning & Feature Engineering

This notebook prepares the datasets collected in Notebook 1 for analysis.  
Key steps include:

- Loading raw datasets (brewery locations, ACS median home values, census tract geometries)
- Cleaning and harmonizing schemas and coordinate reference systems
- Spatial joins to assign breweries to census tracts
- Creating tract-level brewery accessibility metrics using centroid-based distance measures
- Computing brewery counts and densities at the tract level
- Merging cleaned ACS attributes
- Exporting a unified, analysis-ready dataset

Output:  
`data/processed/philly_tracts_final.geojson`

## Setup  
Load packages and custom module functions.
Load datasets created in notebook 1 from local path

In [1]:
import sys
sys.path.append("../src")

import pandas as pd
import geopandas as gpd
import numpy as np
from sklearn.neighbors import KDTree

In [2]:
gdf_brew = gpd.read_file("../data/raw/breweries_philly.geojson")
df_acs = pd.read_csv("../data/raw/acs_philly_median_home_value.csv")
gdf_tracts = gpd.read_file("../data/raw/tracts_philly.geojson")


## Step 1: Clean Brewery Data
- Drop missing geometry
- Keep brewery types relevant to the analysis
- Ensure CRS is correct

In [3]:
gdf_brew = gdf_brew.dropna(subset=["geometry"])
gdf_brew = gdf_brew[gdf_brew["brewery_type"].isin(["micro", "brewpub", "regional"])]

if gdf_brew.crs is None:
    gdf_brew = gdf_brew.set_crs(epsg=4326)
else:
    gdf_brew = gdf_brew.to_crs(epsg=4326)

## Step 2: Clean ACS Median Home Value Data

In [4]:
df_acs["GEOID"] = df_acs["GEOID"].astype(str)
df_acs = df_acs.rename(columns={"median_home_value": "med_home_val"})
df_acs["med_home_val"] = pd.to_numeric(df_acs["med_home_val"], errors="coerce")
df_acs.loc[df_acs["med_home_val"] <= 0, "med_home_val"] = np.nan

## Step 3: Clean Tract Geometries
- Ensure valid geometries
- Harmonize CRS

In [5]:
gdf_tracts = gdf_tracts[gdf_tracts.geometry.notnull()]
gdf_tracts = gdf_tracts[gdf_tracts.is_valid]

if gdf_tracts.crs is None:
    gdf_tracts = gdf_tracts.set_crs(epsg=4326)
else:
    gdf_tracts = gdf_tracts.to_crs(epsg=4326)

gdf_tracts["GEOID"] = gdf_tracts["GEOID"].astype(str)

gdf_tracts = gdf_tracts.merge(df_acs[["GEOID", "med_home_val"]], on="GEOID", how="left")


## Step 5: Brewery Count and density

In [6]:
brew_tract_join = gpd.sjoin(gdf_brew, gdf_tracts, how="left", predicate="within")
brew_counts = brew_tract_join.groupby("GEOID").size()

gdf_tracts["brewery_count"] = gdf_tracts["GEOID"].map(brew_counts).fillna(0).astype(int)
gdf_tracts["area_km2"] = gdf_tracts.to_crs(3857).area / 1e6
gdf_tracts["brewery_density"] = gdf_tracts["brewery_count"] / gdf_tracts["area_km2"]


## Spatial Accessibility Feature

In [7]:
from sklearn.neighbors import KDTree

# Project both layers to meters
tracts_m = gdf_tracts.to_crs(3857).copy()
brews_m  = gdf_brew.to_crs(3857).copy()

# Centroids in meters CRS (safe)
tracts_m["centroid"] = tracts_m.geometry.centroid

# Build nearest-neighbor search tree for breweries
brew_xy = np.column_stack([brews_m.geometry.x, brews_m.geometry.y])
tree = KDTree(brew_xy)

# Query nearest brewery distance for each tract centroid
cent_xy = np.column_stack([tracts_m["centroid"].x, tracts_m["centroid"].y])
dist_m, _ = tree.query(cent_xy, k=1)

# Store back on main GeoDataFrame
gdf_tracts["dist_to_brewery_m"] = dist_m.flatten()
gdf_tracts["dist_to_brewery_km"] = gdf_tracts["dist_to_brewery_m"] / 1000
gdf_tracts["dist_to_brewery_min"] = gdf_tracts["dist_to_brewery_m"] / 80  # ~80m/min

gdf_tracts["brewery_within_800m"] = (gdf_tracts["dist_to_brewery_m"] <= 800).astype(int)
gdf_tracts["brewery_within_1200m"] = (gdf_tracts["dist_to_brewery_m"] <= 1200).astype(int)


## Step 9: Sanity Checks and final dataset cleaning

In [8]:
print(gdf_tracts["med_home_val"].describe())
print("Missing med_home_val:", gdf_tracts["med_home_val"].isna().sum())

print(gdf_tracts["dist_to_brewery_m"].describe())
print(gdf_tracts[["brewery_within_800m", "brewery_within_1200m"]].mean())

count    3.760000e+02
mean     2.619795e+05
std      1.716772e+05
min      4.670000e+04
25%      1.314000e+05
50%      2.227000e+05
75%      3.245250e+05
max      1.036700e+06
Name: med_home_val, dtype: float64
Missing med_home_val: 32
count      408.000000
mean      3880.579448
std       2510.394934
min        102.467807
25%       1816.960671
50%       3363.786585
75%       5651.917913
max      12125.640127
Name: dist_to_brewery_m, dtype: float64
brewery_within_800m     0.063725
brewery_within_1200m    0.137255
dtype: float64


## Finalized Dataset Export

In [9]:
import os
os.makedirs("../data/processed", exist_ok=True)

gdf_final = gdf_tracts.dropna(subset=["med_home_val"]).copy()

gdf_final.to_file("../data/processed/philly_tracts_final.geojson", driver="GeoJSON")
gdf_final.drop(columns="geometry").to_csv("../data/processed/philly_tracts_final.csv", index=False)